In [1]:
"""
advertising_sales_forecast.py
End-to-end pipeline for Advertising dataset (TV, Radio, Newspaper -> Sales)
- EDA (plots, correlation)
- Clean & features
- Linear regression (OLS) + Multiple regression
- RandomForest regression
- Model evaluation (RMSE, MAE, MAPE, R2)
- Advertising elasticity (log-log and finite-diff)
- Scenario: +10% ad spend across channels
- Save plots and model comparison CSV
"""

import os
import sys
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm

# ----------------------------
# Configuration
# ----------------------------
DATA_PATH = r"C:\Users\HP\Sales_Prediction_Project\Advertising.csv"   # update if needed
OUT_DIR = r"C:\Users\HP\Sales_Prediction_Project\advertising_outputs" 
os.makedirs(OUT_DIR, exist_ok=True)
RANDOM_STATE = 42
TEST_SIZE = 0.2

# ----------------------------
# Helper metrics
# ----------------------------
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.where(y_true==0, 1e-9, y_true))) * 100

def print_metrics(name, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{name} -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, MAPE: {mape_val:.2f}%, R2: {r2:.3f}")
    return {'rmse':rmse, 'mae':mae, 'mape':mape_val, 'r2':r2}

def save_fig(fig, fname):
    path = os.path.join(OUT_DIR, fname)
    fig.savefig(path, bbox_inches='tight')
    print("Saved:", path)

# ----------------------------
# 1) Load data
# ----------------------------
if not os.path.exists(DATA_PATH):
    print("ERROR: Data file not found at", DATA_PATH)
    sys.exit(1)

df = pd.read_csv(DATA_PATH)
print("Loaded data shape:", df.shape)
print(df.head())

# Some Advertising CSVs might have spaces or capitals; normalize column names
df.columns = [c.strip() for c in df.columns]

# Expected columns: TV, Radio, Newspaper, Sales
required_cols = ['TV', 'Radio', 'Newspaper', 'Sales']
for c in required_cols:
    if c not in df.columns:
        raise KeyError(f"Required column '{c}' not found in CSV. Columns: {df.columns.tolist()}")

# ----------------------------
# 2) Basic EDA
# ----------------------------
print("\nSummary statistics:")
print(df.describe())

# Pairplot
fig = sns.pairplot(df, diag_kind='kde')
save_fig(plt.gcf(), "pairplot.png")
plt.close()

# Correlation heatmap
corr = df.corr()
fig, ax = plt.subplots(figsize=(6,4))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="RdBu", ax=ax)
ax.set_title("Correlation matrix")
save_fig(fig, "correlation_matrix.png")
plt.close()

# Scatter plots: each channel vs sales
for channel in ['TV','Radio','Newspaper']:
    fig, ax = plt.subplots(figsize=(6,4))
    sns.scatterplot(x=df[channel], y=df['Sales'], ax=ax)
    ax.set_title(f"{channel} vs Sales")
    ax.set_xlabel(channel); ax.set_ylabel("Sales")
    save_fig(fig, f"{channel}_vs_sales.png")
    plt.close()

# ----------------------------
# 3) Data prep / features
# ----------------------------
# No missing values usually, but check
print("\nMissing values per column:\n", df.isna().sum())

# Option: create total_ad_spend
df['Total_Ad'] = df[['TV','Radio','Newspaper']].sum(axis=1)

# Log transforms for elasticity estimation (avoid zeros)
df['log_Sales'] = np.log1p(df['Sales'].clip(lower=0))
df['log_TV'] = np.log1p(df['TV'].clip(lower=0))
df['log_Radio'] = np.log1p(df['Radio'].clip(lower=0))
df['log_Newspaper'] = np.log1p(df['Newspaper'].clip(lower=0))
df['log_Total_Ad'] = np.log1p(df['Total_Ad'].clip(lower=0))

# ----------------------------
# 4) Modeling: simple single-channel + multiple-channel regressions
# ----------------------------
# Prepare X and y
X = df[['TV','Radio','Newspaper']]
y = df['Sales']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# Standard scaling for RF is not necessary, but OK for linear interpretation
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

# 4A) Ordinary Least Squares (statsmodels) - multiple regression (TV + Radio + Newspaper)
X_train_sm = sm.add_constant(X_train)  # NOTE: not scaled for interpretability of coefficients
ols_model = sm.OLS(y_train, X_train_sm).fit()
print("\nOLS summary:")
print(ols_model.summary())

# Predictions OLS
X_test_sm = sm.add_constant(X_test)
y_pred_ols = ols_model.predict(X_test_sm)
metrics_ols = print_metrics("OLS (multiple)", y_test, y_pred_ols)

# 4B) LinearRegression (scikit-learn) for comparison
lr = LinearRegression().fit(X_train_s, y_train)
y_pred_lr = lr.predict(X_test_s)
metrics_lr = print_metrics("LinearRegression (scaled)", y_test, y_pred_lr)

# 4C) RandomForestRegressor
rf = RandomForestRegressor(n_estimators=500, random_state=RANDOM_STATE).fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
metrics_rf = print_metrics("RandomForest", y_test, y_pred_rf)

# Save model comparison
comparison = pd.DataFrame([
    {'model':'OLS','rmse':metrics_ols['rmse'],'mae':metrics_ols['mae'],'mape':metrics_ols['mape'],'r2':metrics_ols['r2']},
    {'model':'LinearRegression','rmse':metrics_lr['rmse'],'mae':metrics_lr['mae'],'mape':metrics_lr['mape'],'r2':metrics_lr['r2']},
    {'model':'RandomForest','rmse':metrics_rf['rmse'],'mae':metrics_rf['mae'],'mape':metrics_rf['mape'],'r2':metrics_rf['r2']}
])
comp_path = os.path.join(OUT_DIR, "model_comparison.csv")
comparison.to_csv(comp_path, index=False)
print("Saved model comparison to:", comp_path)

# ----------------------------
# 5) Residual plot & actual vs predicted
# ----------------------------
def plot_actual_vs_pred(y_true, preds_dict, title_suffix=""):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(y_true.values, label='Actual', marker='o')
    for name, yp in preds_dict.items():
        ax.plot(yp, label=name, marker='x')
    ax.set_title("Actual vs Predicted Sales " + title_suffix)
    ax.set_xlabel("Test sample index")
    ax.set_ylabel("Sales")
    ax.legend()
    save_fig(fig, f"actual_vs_pred{title_suffix.replace(' ','_')}.png")
    plt.close(fig)

plot_actual_vs_pred(y_test.reset_index(drop=True), {
    'OLS': np.array(y_pred_ols).reshape(-1),
    'LinearRegression': np.array(y_pred_lr).reshape(-1),
    'RandomForest': np.array(y_pred_rf).reshape(-1)
}, title_suffix="(test set)")

# Residual histogram for best model (choose by RMSE)
best_model_name = comparison.sort_values('rmse').iloc[0]['model']
print("Best model by RMSE:", best_model_name)

if best_model_name == 'RandomForest':
    residuals = y_test.values - y_pred_rf
elif best_model_name == 'OLS':
    residuals = y_test.values - y_pred_ols
else:
    residuals = y_test.values - y_pred_lr

fig, ax = plt.subplots(figsize=(6,4))
sns.histplot(residuals, kde=True, ax=ax)
ax.set_title(f"Residuals histogram - {best_model_name}")
save_fig(fig, f"residuals_{best_model_name}.png")
plt.close(fig)

# ----------------------------
# 6) Advertising elasticity analysis
# Method A: log-log regression on total ad (single elasticity estimate)
# Method B: finite-difference scenario using RandomForest (or best model)
# ----------------------------
# --- Method A: OLS on logs (log_total_ad -> log_sales)
X_log = sm.add_constant(df[['log_Total_Ad']])
y_log = df['log_Sales']
log_ols = sm.OLS(y_log, X_log).fit()
print("\nLog-log OLS summary (Total ad -> Sales):")
print(log_ols.summary())
# elasticity estimate:
elasticity_total = log_ols.params.get('log_Total_Ad', None)
print("Estimated total-ad elasticity (log-log):", elasticity_total)

# --- Method B: approximate marginal effect per channel using finite diff on RF
# We'll treat RandomForest as the predictive engine and compute average pct-change when increasing each channel by 10%
def channel_uplift_pct(model, X_df, channel, pct=0.10):
    """
    model: trained model that accepts raw channel columns in the same order as X_df[['TV','Radio','Newspaper']]
    X_df: dataframe of predictors
    channel: one of 'TV','Radio','Newspaper' or 'Total_Ad'
    pct: fractional increase (0.10 = +10%)
    Returns average percent change in predicted sales.
    """
    X_base = X_df[['TV','Radio','Newspaper']].copy().reset_index(drop=True)
    # baseline pred
    pred_base = model.predict(X_base)
    X_up = X_base.copy()
    if channel == 'Total_Ad':
        # proportionally increase each channel so total ad increases by pct
        X_up = X_up * (1 + pct)
    else:
        X_up[channel] = X_up[channel] * (1 + pct)
    pred_up = model.predict(X_up)
    avg_pct_change = (pred_up.mean() - pred_base.mean()) / pred_base.mean() * 100
    return avg_pct_change

# Compute +10% uplift per channel using RF
uplift_tv = channel_uplift_pct(rf, df, 'TV', pct=0.10)
uplift_radio = channel_uplift_pct(rf, df, 'Radio', pct=0.10)
uplift_newspaper = channel_uplift_pct(rf, df, 'Newspaper', pct=0.10)
uplift_total = channel_uplift_pct(rf, df, 'Total_Ad', pct=0.10)

print(f"\n+10% TV -> avg sales change (RF): {uplift_tv:.2f}%")
print(f"+10% Radio -> avg sales change (RF): {uplift_radio:.2f}%")
print(f"+10% Newspaper -> avg sales change (RF): {uplift_newspaper:.2f}%")
print(f"+10% Total_Ad -> avg sales change (RF): {uplift_total:.2f}%")

# Save elasticity table
elasticity_df = pd.DataFrame([
    {'method':'log-log_total','channel':'Total_Ad','elasticity':elasticity_total},
    {'method':'rf_uplift_pct','channel':'TV','pct_change':uplift_tv},
    {'method':'rf_uplift_pct','channel':'Radio','pct_change':uplift_radio},
    {'method':'rf_uplift_pct','channel':'Newspaper','pct_change':uplift_newspaper},
    {'method':'rf_uplift_pct','channel':'Total_Ad','pct_change':uplift_total},
])
elasticity_df.to_csv(os.path.join(OUT_DIR, "elasticity_estimates.csv"), index=False)
print("Saved elasticity estimates to:", os.path.join(OUT_DIR, "elasticity_estimates.csv"))

# ----------------------------
# 7) Scenario: budget reallocation or +10% ad spend overall
# Example 1: +10% on Total_Ad (proportionally across channels)
# Example 2: move $X from Newspaper to TV (simple simulation)
# ----------------------------
# baseline prediction average
baseline_pred = rf.predict(df[['TV','Radio','Newspaper']])
print("\nAverage baseline predicted sales (RF):", baseline_pred.mean())

# Scenario A: +10% total ad (proportional increase)
df_up = df[['TV','Radio','Newspaper']].copy() * 1.10
pred_up_total = rf.predict(df_up)
avg_pct_total = (pred_up_total.mean() - baseline_pred.mean()) / baseline_pred.mean() * 100
print("Scenario A (+10% total ad proportional) -> avg sales change (RF): {:.2f}%".format(avg_pct_total))

# Scenario B: shift 20% of Newspaper budget to TV (no change in total)
df_shift = df[['TV','Radio','Newspaper']].copy()
shift_amount = 0.20 * df_shift['Newspaper']  # 20% of newspaper spends
df_shift['Newspaper'] = df_shift['Newspaper'] - shift_amount
df_shift['TV'] = df_shift['TV'] + shift_amount
pred_shift = rf.predict(df_shift)
avg_pct_shift = (pred_shift.mean() - baseline_pred.mean()) / baseline_pred.mean() * 100
print("Scenario B (shift 20% Newspaper -> TV) -> avg sales change (RF): {:.2f}%".format(avg_pct_shift))

# Save scenario results
pd.DataFrame([
    {'scenario':'baseline','avg_pred':baseline_pred.mean()},
    {'scenario':'+10%_total','avg_pred':pred_up_total.mean(),'pct_change':avg_pct_total},
    {'scenario':'shift_20pct_newspaper_to_tv','avg_pred':pred_shift.mean(),'pct_change':avg_pct_shift},
]).to_csv(os.path.join(OUT_DIR, "scenario_results.csv"), index=False)
print("Saved scenario results to:", os.path.join(OUT_DIR, "scenario_results.csv"))

# ----------------------------
# 8) Feature importances (RF) and partial dependence (simple)
# ----------------------------
fi = pd.Series(rf.feature_importances_, index=['TV','Radio','Newspaper']).sort_values(ascending=True)
fig, ax = plt.subplots(figsize=(6,3))
fi.plot(kind='barh', ax=ax)
ax.set_title("RF Feature Importances")
save_fig(fig, "rf_feature_importances.png")
plt.close(fig)

# Simple PDP-style scatter: TV vs predicted sales (keeping others fixed at median)
tv_vals = np.linspace(df['TV'].min(), df['TV'].max(), 50)
pdp_df = pd.DataFrame({
    'TV': tv_vals,
    'Radio': np.repeat(df['Radio'].median(), len(tv_vals)),
    'Newspaper': np.repeat(df['Newspaper'].median(), len(tv_vals))
})
pdp_preds = rf.predict(pdp_df)
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(tv_vals, pdp_preds, marker='o')
ax.set_xlabel("TV spend")
ax.set_ylabel("Predicted Sales")
ax.set_title("Partial dependence-like plot (TV)")
save_fig(fig, "pdp_tv.png")
plt.close(fig)

# ----------------------------
# 9) Final recommendations (printed) + saved summary
# ----------------------------
print("\n--- Summary Recommendations ---")
print("1) TV usually shows the strongest effect (check feature importances & log-OLS coefficient).")
print("2) Use per-channel elasticity (from RF uplift or log-log regressions) to evaluate marginal ROI.")
print("3) Before large budget changes, run small controlled lifts (A/B or geo experiments) to confirm model estimates.")
print("4) Consider shifting budget from low-return channels (e.g., Newspaper) to high-return channels (e.g., TV or Radio) if scenario simulations show uplift.")
print("5) Retrain models periodically and include seasonality/time variables if you have a time series dataset with dates.")

# Save textual summary
with open(os.path.join(OUT_DIR, "readme_summary.txt"), "w") as f:
    f.write("Advertising -> Sales pipeline outputs\n")
    f.write("Files: pairplot.png, correlation_matrix.png, *_vs_sales.png, actual_vs_pred.png, rf_feature_importances.png, pdp_tv.png, model_comparison.csv, elasticity_estimates.csv, scenario_results.csv\n")
    f.write("\nRecommendations:\n1) Use elasticity to guide spend changes.\n2) Validate via experiments.\n3) Favor channels with higher marginal returns.\n")
print("Saved summary to:", os.path.join(OUT_DIR, "readme_summary.txt"))

print("\n--- Pipeline complete. Outputs in:", OUT_DIR)


Loaded data shape: (200, 5)
   Unnamed: 0     TV  Radio  Newspaper  Sales
0           1  230.1   37.8       69.2   22.1
1           2   44.5   39.3       45.1   10.4
2           3   17.2   45.9       69.3    9.3
3           4  151.5   41.3       58.5   18.5
4           5  180.8   10.8       58.4   12.9

Summary statistics:
       Unnamed: 0          TV       Radio   Newspaper       Sales
count  200.000000  200.000000  200.000000  200.000000  200.000000
mean   100.500000  147.042500   23.264000   30.554000   14.022500
std     57.879185   85.854236   14.846809   21.778621    5.217457
min      1.000000    0.700000    0.000000    0.300000    1.600000
25%     50.750000   74.375000    9.975000   12.750000   10.375000
50%    100.500000  149.750000   22.900000   25.750000   12.900000
75%    150.250000  218.825000   36.525000   45.100000   17.400000
max    200.000000  296.400000   49.600000  114.000000   27.000000
Saved: C:\Users\HP\Sales_Prediction_Project\advertising_outputs\pairplot.png
Save